# Advanced PAHFIT tricks

Main goal of this notebook: share some of the tweaks I typically apply to the science pack, the input data, the fitting.
You can experiment with these for particularly stubborn fitting problems. 

Examples include
1. Dealing with "non-finite value" errors: remove zero uncertainties, prevent very small uncertainties, remove large positive/negative spikes in flux.
2. Fits that don't seem to converge: add fudge term to the uncertainty to avoid bad biases, avoid initial guesses near a local minimum
3. Faster fitting + additional way to deal with conversion issues: remove gas lines from both spectrum and model, and fit them separately.

Some useful code that is not in the standard PAHFIT (but used in this notebook) can be found in pahfit_hacks.py and plot_decomposition.py.

In [ ]:
# when ipympl is installed, enable interactive plots like this
import matplotlib
%matplotlib widget
import os
from specutils import Spectrum1D
from astropy.table import Table 
from astropy import units as u
from astropy.nddata import StdDevUncertainty
from matplotlib import pyplot as plt
import numpy as np
from pahfit.model import Model
from specutils import Spectrum1D
from plot_decomposition import plot_decomposition
from pahfit_hacks import remove_lines_from_data_and_model

## Load PDRs4All template spectrum data as example

In [ ]:
t = Table.read('pdrs4all_templates_seg3_crds1084_20230514.dat', 
               data_start=7, format='ascii', 
               names=["Wave", "seg",  "flux T1","error T1", "flux T2", "error T2", "flux T3", "error T3", "flux T4", "error T4", "flux T5", "error T5"])

## Cutting the data

- Avoid data with bad uncertainties (zero is very bad! fit will crash)
- Avoid data with spikes in the flux (speaks for itself. Always clean your data!)
- Other problems you might know about in your data. E.g. remove other artifacts such as weird continuum. PAHFIT only has 'physically motivated' continua, so the result can be biased by local continuum-type artifacts.

In [ ]:
tgood = t[t['flux T3'] > 0]
spec = Spectrum1D(tgood['flux T3'] * u.MJy / u.sr, tgood['Wave'] * u.micron, uncertainty=StdDevUncertainty((tgood['error T3'] * u.MJy / u.sr)))

## Double check for unreasonably high signal to noise

Typically, if there are points with SNR > 100, one should be worried because those might bias the $\chi^2$ and get it stuck in a local minimum. In that case, it can be good to add a fudge term that effectively caps the SNR at 100.

In [ ]:
plt.figure()
plt.subplot(121)
plt.plot(spec.wavelength, np.abs(spec.flux.value) / spec.uncertainty.array)
plt.subplot(122)
plt.hist(spec.uncertainty.array)
plt.xlabel('uncertainty')
print("counted zero uncertainties: ", np.count_nonzero(spec.uncertainty.array == 0))
print("counted SNR > 50: ", np.count_nonzero(spec.uncertainty.array * 50 < spec.flux.value))

## Fit without lines

The difference between lines and broad features can be trouble sometimes for the minimizer. As an added bonus, fitting without lines is much faster. Another good idea: use the results of this fit without lines as an initial guess.

In [ ]:
instrumentname = 'jwst.nirspec.g395.high'
packfile = "classic.yaml"
spec.meta['instrument'] = 'jwst.nirspec.g395.high'
spec_nolines, model_nolines = remove_lines_from_data_and_model(spec, Model.from_yaml(packfile))
model_nolines.guess(spec_nolines)
model_nolines.fit(spec_nolines)

In [ ]:
_ = plot_decomposition(spec_nolines, model_nolines)

## Fit with lines using previous result as initial guess

First, load a default model (the full one). Then, instead of applying guess(), copy the values from the nolines model instead. Plot this guess to see if it's working.

In [ ]:
spec.meta['instrument'] = instrumentname
# redshift = 0.0
# spec.set_redshift_to(redshift)

In [ ]:
model = Model.from_yaml(packfile)
# model.guess(spec)
for row in model_nolines.features:
    model.features.loc[row['name']] = row

_ = plot_decomposition(spec, model)

Now run the fit. Both the lines and the other features will be adjusted. An even better approach, might be to freeze the dust and continuum features, and only let the gas lines vary. Requires some hacking of the 'power' column in the features table, but is technically possible.

In [ ]:
model.fit(spec, maxiter=10000)
_ = plot_decomposition(spec, model)